In [1]:
# imports
from importlib import reload

import os # a module to interact with the operating system
os.environ["ENDPOINT_URL"]="http://rook-ceph-rgw-nautiluss3.rook"
import numpy as np

import pandas
import xarray
import h5py
import healpy as hp
import time

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats

from ulmo import io as ulmo_io
from ulmo import plotting
from ulmo.llc import io as llc_io
from ulmo.llc import plotting as llc_plotting

from ulmo.utils import image_utils
from ulmo.analysis import figures as ulmo_figs
from ulmo.plotting import plotting as ulmo_plotting
from ulmo.analysis import spatial_plots as sp

ModuleNotFoundError: No module named 'xarray'

# Load tables

In [ ]:
os.getenv('ENDPOINT_URL') 
llc_uniform= ulmo_io.load_main_table( 's3://llc/Tables/test_uniform_r0.5_test.feather' )
llc_uniform.head()

In [ ]:
viirs_tbl = ulmo_io.load_main_table( 's3://viirs/Tables/VIIRS_all_99clear_std.parquet')

# Work on functions

In [ ]:
eval_tbl = llc_uniform
nside = 64

In [ ]:
# Grab lats, lons
lats = eval_tbl.lat.values
lons = eval_tbl.lon.values

# Grab LL values
vals = eval_tbl.LL.values

# Healpix coords
theta = (90 - lats) * np.pi / 180.  # convert into radians
phi = lons * np.pi / 180.
idx_all = hp.pixelfunc.ang2pix(nside, theta, phi) # returns the healpix pixel numbers that correspond to theta and phi values

# Count events
npix_hp = hp.nside2npix(nside)  # returns the number of pixels on map, based on nside parameter

### Look at idx_all

In [ ]:
idx_all.shape[0] == eval_tbl.shape[0]

In [ ]:
idx_series_RD = pd.Series(idx_all)
idx_series_RD

In [ ]:
idx_series = idx_series_RD.sort_values()
idx_series

In [ ]:
pixels = pd.unique(idx_series)

### Create a subroutine: for a unique pixel, gather all LL values and then take the median

In [ ]:
pixel = pixels[0]

In [ ]:
import time

In [ ]:
meds = np.ma.masked_array(np.zeros(npix_hp, dtype='float'))
 
start = time.time()

# find where which cutouts to put in that pixel
where = np.where(pixel == idx_series)
first = where[0][0]
last = where[0][-1]
indices = idx_series[first:last + 1].index

# evaluate the median LL value for that pixel 
vals = eval_tbl.iloc[indices.to_numpy()].LL.to_numpy()

meds[pixel] = np.median( vals )
end = time.time()   

In [ ]:
end -start

#### Should take about 3 minutes for all

### Iterate this subroutine for all pixels

In [ ]:
pixels = pd.unique(pd.Series(idx_all).sort_values())
meds = np.ma.masked_array(np.zeros(npix_hp, dtype='float'))

for pixel in pixels: 
    
    # find where which cutouts to put in that pixel
    where = np.where(pixel == idx_series)
    first = where[0][0]
    last = where[0][-1]
    indices = idx_series[first:last + 1].index
    
    # evaluate the median LL value for that pixel 
    vals = eval_tbl.iloc[indices.to_numpy()].LL.to_numpy()
    
    meds[pixel] = np.median( vals )
    

In [ ]:
idx_series

### Incorporate into the function

In [ ]:
def evals_to_healpix_meds(eval_tbl, nside,  mask=True):
    """
    Generate a healpix map of where the input
    MHW Systems are located on the globe

    Parameters
    ----------
    mhw_sys : pandas.DataFrame
    nside : int  # nside is a number that sets the resolution of map
    mask : bool, optional

    Returns
    -------
    num of events, lats, lons, median values : hp.ma, np.ndarray, np.ndarray, hp.ma

    """
    # Grab lats, lons
    lats = eval_tbl.lat.values
    lons = eval_tbl.lon.values

    # Grab LL values
    vals = eval_tbl.LL.values

    # Healpix coords
    theta = (90 - lats) * np.pi / 180.  # convert into radians
    phi = lons * np.pi / 180.
    idx_all = hp.pixelfunc.ang2pix(nside, theta, phi) # returns the healpix pixel numbers that correspond to theta and phi values

    # Intialize the arrays
    npix_hp = hp.nside2npix(nside)  # returns the number of pixels on map, based on nside parameter
    all_events = np.ma.masked_array(np.zeros(npix_hp, dtype='int')) # array of all pixels on map
    med_values = np.ma.masked_array(np.zeros(npix_hp, dtype='float')) # will contain median LL value in that pixel

    # Count events
    for i, idx in enumerate(idx_all):
        all_events[idx] += 1 # pixels concentrated with data pts >= 1 ; those without data remain 0

    zero = all_events == 0 
    float_events = all_events.astype(float)
# ~ operator is called the complement bitwise operator 
# inverts the True/False values
# [~zero] selects pixels where the cutouts are (where events = 1 exist)


    # Calculate median values
    idx_arr = pandas.Series(idx_all).sort_values()
    pixels = pandas.unique(idx_arr)

    for pixel in pixels: 
    
        # find where which cutouts to put in that pixel
        where = np.where(pixel == idx_arr)
        first = where[0][0]
        last = where[0][-1]
        indices = idx_arr[first:last + 1].index
    
        # evaluate the median LL value for that pixel 
        vals = eval_tbl.iloc[indices.to_numpy()].LL.to_numpy()
    
        med_values[pixel] = np.median( vals )


    # Mask
    evts = hp.ma(float_events)
    meds = hp.ma(med_values)
    if mask:  # if you want to mask float_events
        evts.mask = zero # current mask set to zero array, where Trues (no events) are masked
        meds.mask = zero 

    # Angles
    hp_lons, hp_lats = hp.pixelfunc.pix2ang(nside, np.arange(npix_hp), lonlat=True)

    # Return
    return evts, hp_lons, hp_lats, meds

In [ ]:
def show_med_LL(main_tbl:pandas.DataFrame, 
                 nside=64, 
                 use_mask=True, tricontour=False,
                 lbl=None, figsize=(12,8), 
                 color='viridis', show=True):
    """Generate a global map of the location of the input
    cutouts
    Args:
        main_tbl (pandas.DataFrame): table of cutouts
        nside (int, optional): [description]. Defaults to 64.
        use_log (bool, optional): [description]. Defaults to True.
        use_mask (bool, optional): [description]. Defaults to True.
        tricontour (bool, optional): [description]. Defaults to False.
        lbl ([type], optional): [description]. Defaults to None.
        figsize (tuple, optional): [description]. Defaults to (12,8).
        color (str, optional): [description]. Defaults to 'Reds'.
        show (bool, optional): If True, show on the screen.  Defaults to True
    Returns:
        matplotlib.Axis: axis holding the plot
    """
    # Healpix me
    hp_events, hp_lons, hp_lats, hp_values = evals_to_healpix_meds(
        main_tbl, nside, mask=use_mask)
    
    # Figure
    
    fig = plt.figure(figsize=figsize)
    plt.clf()

    tformM = ccrs.Mollweide()
    tformP = ccrs.PlateCarree()

    ax = plt.axes(projection=tformM)

    if tricontour:
        cm = plt.get_cmap(color)
        img = ax.tricontourf(hp_lons, hp_lats, hp_values, transform=tformM,
                         levels=20, cmap=cm)#, zorder=10)
    else:
        cm = plt.get_cmap(color)
        # Cut
        good = np.invert(hp_values.mask)
        img = plt.scatter(x=hp_lons[good],
            y=hp_lats[good],
            c=hp_values[good], vmin = -1000, vmax = 500, 
            cmap=cm,
            s=1,
            transform=tformP)

    # Colorbar
    cb = plt.colorbar(img, orientation='horizontal', pad=0.)
    if lbl is not None:
        clbl = 'mean_LL'
        cb.set_label(clbl, fontsize=20.)
    cb.ax.tick_params(labelsize=17)

    # Coast lines
    if not tricontour:
        ax.coastlines(zorder=10)
        ax.set_global()
    
        gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, 
            color='black', alpha=0.5, linestyle=':', draw_labels=True)
        gl.xlabels_top = False
        gl.ylabels_left = True
        gl.ylabels_right=False
        gl.xlines = True
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'color': 'black'}# 'weight': 'bold'}
        gl.ylabel_style = {'color': 'black'}# 'weight': 'bold'}
        #gl.xlocator = mticker.FixedLocator([-180., -160, -140, -120, -60, -20.])
        #gl.xlocator = mticker.FixedLocator([-240., -180., -120, -65, -60, -55, 0, 60, 120.])
        #gl.ylocator = mticker.FixedLocator([0., 15., 30., 45, 60.])


    # Layout and save
    if show:
        plt.show()

    return ax

In [ ]:
show_med_LL( llc_uniform) 

In [ ]:
def show_spatial_two_med(tbl1:pandas.DataFrame, tbl2:pandas.DataFrame, 
                 nside=64, 
                 use_mask=True, tricontour=False,
                 lbl=None, figsize=(12,8), 
                 color='coolwarm', show=True):
    """Generate a global map of the location of the input
    cutouts

    Args:
        main_tbl (pandas.DataFrame): table of cutouts
        nside (int, optional): [description]. Defaults to 64.
        use_log (bool, optional): [description]. Defaults to True.
        use_mask (bool, optional): [description]. Defaults to True.
        tricontour (bool, optional): [description]. Defaults to False.
        lbl ([type], optional): [description]. Defaults to None.
        figsize (tuple, optional): [description]. Defaults to (12,8).
        color (str, optional): [description]. Defaults to 'Reds'.
        show (bool, optional): If True, show on the screen.  Defaults to True

    Returns:
        matplotlib.Axis: axis holding the plot
    """
    # Healpix me
    hp_events1, hp_lons1, hp_lats1, hp_values1 = sp.evals_to_healpix_meds(
        tbl1, nside, mask=use_mask)
    
    hp_events2, hp_lons2, hp_lats2, hp_values2 = sp.evals_to_healpix_meds(
        tbl2, nside, mask=use_mask)
    
    # Figure
    
    fig = plt.figure(figsize=figsize)
    plt.clf()

    tformM = ccrs.Mollweide()
    tformP = ccrs.PlateCarree()

    ax = plt.axes(projection=tformM)

    if tricontour:
        cm = plt.get_cmap(color)
        img = ax.tricontourf(hp_lons1, hp_lats1, hp_values1 - hp_values2, transform=tformM,
                         levels=20, cmap=cm)#, zorder=10)
    else:
        cm = plt.get_cmap(color)
        # Cut
        good = np.invert(hp_values2.mask)
        img = plt.scatter(x=hp_lons2[good],
            y=hp_lats2[good],
            c=hp_values1[good]- hp_values2[good], vmin = -300, vmax = 300, 
            cmap=cm,
            s=100,
            transform=tformP)

    # Colorbar
    cb = plt.colorbar(img, orientation='horizontal', pad=0.1)
    if lbl is not None:
        #clbl=r'$\log_{10} \, N_{\rm '+'{}'.format(lbl)+'}$'
        clbl = r'$LL_{VIIRS} - LL_{LLC}$'
        cb.set_label(clbl, fontsize=20.)
    cb.ax.tick_params(labelsize=17)
    
    # Rectangle: highlight region of interest
    
    xrange = np.arange(-105,-94,1)
    yrange = np.arange(-2, 3, 1)
    
    plt.plot(xrange,2*np.ones(11), 'k')
    plt.plot(xrange,-2*np.ones(11), 'k')
    plt.plot(-105*np.ones(5), yrange, 'k')
    plt.plot(-95*np.ones(5), yrange, 'k')

    # Coast lines
    if not tricontour:
        ax.coastlines(zorder=10)
        ax.set_extent([-120, -80, -10, 10], ccrs.PlateCarree())
    
        gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, x_inline=False, y_inline=False, 
        color='black', alpha=0.5, linestyle=':', draw_labels=True)
        gl.top_labels=False
        gl.bottom_labels=True
        gl.left_labels=True
        gl.right_labels=False
        gl.xlines = True
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'color': 'black', 'size': 14, 'weight': 'bold'}
        gl.ylabel_style = {'color': 'black', 'size': 14, 'weight': 'bold'}
        #gl.xlocator = mticker.FixedLocator([-180., -160, -140, -120, -60, -20.])
        #gl.xlocator = mticker.FixedLocator([-240., -180., -120, -65, -60, -55, 0, 60, 120.])
        #gl.ylocator = mticker.FixedLocator([0., 15., 30., 45, 60.])

        plt.savefig('med_LL_diff_VIIRS_vs_LLC', dpi = 300)

    # Layout and save
    if show:
        plt.show()

    return ax

In [ ]:
xrange = np.arange(-105,-94,1)
xrange.shape
yrange = np.arange(-2, 3, 1)
yrange.shape
np.ones(4).shape

In [ ]:
xrange

In [ ]:
plt.plot(xrange,2*np.ones(11), 'k')
plt.plot(xrange,-2*np.ones(11), 'k')
plt.plot(-105*np.ones(5), yrange, 'k')
plt.plot(-95*np.ones(5), yrange, 'k')
plt.show()

In [ ]:
def show_med_LL1(main_tbl:pandas.DataFrame, 
                 nside=64, 
                 use_mask=True, tricontour=False,
                 lbl=None, figsize=(12,8), 
                 color='viridis', show=True):
    """Generate a global map of the location of the input
    cutouts
    Args:
        main_tbl (pandas.DataFrame): table of cutouts
        nside (int, optional): [description]. Defaults to 64.
        use_log (bool, optional): [description]. Defaults to True.
        use_mask (bool, optional): [description]. Defaults to True.
        tricontour (bool, optional): [description]. Defaults to False.
        lbl ([type], optional): [description]. Defaults to None.
        figsize (tuple, optional): [description]. Defaults to (12,8).
        color (str, optional): [description]. Defaults to 'Reds'.
        show (bool, optional): If True, show on the screen.  Defaults to True
    Returns:
        matplotlib.Axis: axis holding the plot
    """
    # Healpix me
    hp_events, hp_lons, hp_lats, hp_values = evals_to_healpix_meds(
        main_tbl, nside, mask=use_mask)
    
    # Figure
    
    fig = plt.figure(figsize=figsize)
    plt.clf()

    tformM = ccrs.Mollweide()
    tformP = ccrs.PlateCarree()

    ax = plt.axes(projection=tformM)

    
    cm = plt.get_cmap(color)
    # Cut
    good = np.invert(hp_values.mask)
    img = plt.scatter(x=hp_lons[good],
        y=hp_lats[good],
        c=hp_values[good], vmin = -1000, vmax = 500, 
        cmap=cm,
        s=2500,
        transform=tformP)

    # Colorbar
    cb = plt.colorbar(img, orientation='horizontal', pad=0.1)
    if lbl is not None:
        clbl = 'mean_LL'
        cb.set_label(clbl, fontsize=20.)
    cb.ax.tick_params(labelsize=17)

    # Coast lines
    ax.coastlines(zorder=10)
    ax.set_extent([-115, -95, -2, 2], ccrs.PlateCarree())
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, 
        color='black', alpha=0.5, linestyle=':', draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right=False
    gl.xlines = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'color': 'black','weight': 'bold'}
    gl.ylabel_style = {'color': 'black', 'weight': 'bold'}
        #gl.xlocator = mticker.FixedLocator([-180., -160, -140, -120, -60, -20.])
        #gl.xlocator = mticker.FixedLocator([-240., -180., -120, -65, -60, -55, 0, 60, 120.])
        #gl.ylocator = mticker.FixedLocator([0., 15., 30., 45, 60.])


    # Layout and save
    if show:
        plt.show()

    return ax

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.clf()

tformM = ccrs.Mollweide()
tformP = ccrs.PlateCarree()

ax = plt.axes(projection=tformM)
    
ax.coastlines(zorder=10)
ax.set_extent([-105, -95, -2, 2], ccrs.PlateCarree())
    
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, x_inline=False, y_inline=False, 
        color='black', alpha=0.5, linestyle=':', draw_labels=True)
gl.top_labels=False
gl.bottom_labels=True
gl.left_labels=True
gl.right_labels=False
#gl.xformatter = LONGITUDE_FORMATTER
#gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'color': 'black','weight': 'bold'}
gl.ylabel_style = {'color': 'black', 'weight': 'bold'}

cm = plt.get_cmap('viridis')

img = plt.scatter(x=-100,
            y=0,
            c=-200, vmin = -300, vmax = 300, 
            cmap=cm,
            s=2500,
            transform=tformP)

In [ ]:
show_spatial_two_med(viirs_tbl, llc_uniform, lbl=True)

In [ ]:
eqtr_n = (llc_uniform.lat > 0. ) & (np.abs(llc_uniform.lat) < 2.) & (np.abs(llc_uniform.lon + 100) < 5.)
llc_eqtr_n = llc_uniform[ eqtr_n ]

med_LL_n = np.median(llc_eqtr_n.LL.to_numpy())
med_LL_n

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


ax = plt.axes(projection=ccrs.Mercator())
ax.coastlines()

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_left = False
gl.xlines = False
gl.xlocator = mticker.FixedLocator([-180, -45, 0, 45, 180])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.xlabel_style = {'color': 'red', 'weight': 'bold'}

plt.show()